In [2]:
import requests
import json
import re

def fetch_all_products():
    """Fetches 100 products from DummyJSON API"""
    # Requirement: Fetch all available products using limit=100
    url = 'https://dummyjson.com/products?limit=100'
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        print(f"Successfully fetched {len(data['products'])} products from API.")
        return data['products']
    except Exception as e:
        # Requirement: Return empty list if API fails
        print(f"API Failure: {e}")
        return []

def create_product_mapping(api_products):
    """Creates a mapping of product IDs to product info"""
    mapping = {}
    for item in api_products:
        mapping[item['id']] = {
            'title': item.get('title'),
            'category': item.get('category'),
            'brand': item.get('brand'),
            'rating': item.get('rating')
        }
    return mapping

def enrich_sales_data(transactions, product_mapping):
    """Enriches transaction data with API product information"""
    enriched_list = []
    
    for tx in transactions:
        # Requirement: Extract numeric ID from ProductID string (e.g., P1 -> 1)
        raw_id = tx.get('ProductID', '')
        numeric_id_match = re.search(r'\d+', str(raw_id))
        
        enriched_tx = tx.copy()
        
        if numeric_id_match:
            prod_id = int(numeric_id_match.group())
            
            if prod_id in product_mapping:
                info = product_mapping[prod_id]
                enriched_tx['API_Category'] = info['category']
                enriched_tx['API_Brand'] = info['brand']
                enriched_tx['API_Rating'] = info['rating']
                enriched_tx['API_Match'] = True
            else:
                enriched_tx.update({
                    'API_Category': None, 'API_Brand': None, 
                    'API_Rating': None, 'API_Match': False
                })
        else:
            enriched_tx.update({
                'API_Category': None, 'API_Brand': None, 
                'API_Rating': None, 'API_Match': False
            })
            
        enriched_list.append(enriched_tx)
        
    return enriched_list

def save_enriched_data(enriched_transactions, filename='data/enriched_sales_data.txt'):
    """Saves enriched transactions back to file using pipe delimiter"""
    if not enriched_transactions:
        return
        
    # Requirement: Create output file with original + new fields
    header = "TransactionID|Date|ProductID|ProductName|Quantity|UnitPrice|CustomerID|Region|API_Category|API_Brand|API_Rating|API_Match"
    
    try:
        with open(filename, 'w') as f:
            f.write(header + "\n")
            for tx in enriched_transactions:
                line = [
                    str(tx.get('TransactionID', '')), str(tx.get('Date', '')),
                    str(tx.get('ProductID', '')), str(tx.get('ProductName', '')),
                    str(tx.get('Quantity', '')), str(tx.get('UnitPrice', '')),
                    str(tx.get('CustomerID', '')), str(tx.get('Region', '')),
                    str(tx.get('API_Category', 'None')), str(tx.get('API_Brand', 'None')),
                    str(tx.get('API_Rating', 'None')), str(tx.get('API_Match', 'False'))
                ]
                f.write("|".join(line) + "\n")
        print(f"Data successfully saved to {filename}")
    except Exception as e:
        print(f"Error saving file: {e}")

In [3]:
def process_enrichment_summary(enriched_data):
    df = pd.DataFrame(enriched_data)
    df["API_Match"] = df["API_Match"].astype(bool)

    # CHANGE THESE LINES: Use len(df) or TransactionID to count records
    total_processed = len(df) 
    successful = len(df[df["API_Match"] == True])
    failed = len(df[df["API_Match"] == False])
    
    # Calculate rate based on total records
    success_rate = (successful / total_processed) * 100 if total_processed > 0 else 0
    # Get unique product names that failed enrichment
    not_enriched_products = df[df["API_Match"] == False]["ProductName"].unique().tolist()
   
    print("API ENRICHMENT SUMMARY")
    print("----------------------------\n")
    print(f"Total Products Processed: {total_records}")
    print(f"Successful Enrichments: {successful}")
    print(f"Failed Enrichments: {failed}")
    print(f"Success Rate: {success_rate:.2f}%\n")
    print("Products Not Enriched:")
    if not_enriched_products:
        for product in not_enriched_products:
            print(f"- {product}")
    else:
        print("- None")